<a href="https://colab.research.google.com/github/JaCaRego/JaCaRego/blob/main/Limpeza_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Resumo do Código

Este notebook realiza as seguintes etapas de limpeza e preparação de dados em um DataFrame Pandas lido do arquivo 'clientes.csv':

1.  **Configuração de exibição do Pandas**: Ajusta as opções de exibição para mostrar todas as colunas, linhas e largura de coluna.
2.  **Visualização inicial dos dados**: Exibe as primeiras e últimas linhas, além de um resumo estatístico (`df.head()`, `df.tail()`, `df.describe()`).
3.  **Remoção de dados**: Remove a coluna 'pais' e a terceira linha (índice 2) do DataFrame.
4.  **Normalização de texto**: Converte a coluna 'nome' para título, 'endereco' para minúsculas e 'estado' para maiúsculas e remove espaços em branco.
5.  **Conversão de tipo de dados**: Converte a coluna 'idade' para o tipo inteiro.
6.  **Tratamento de valores nulos (NaN)**:
    *   Cria DataFrames com valores nulos substituídos por 0 (`df_fillna`) e com linhas que contêm nulos removidas (`df_dropna`).
    *   Cria um DataFrame mantendo apenas linhas com pelo menos 4 valores não nulos (`df_dropna4`).
    *   Remove linhas onde a coluna 'cpf' é nula.
    *   Preenche nulos na coluna 'estado' com 'Desconhecido', na coluna 'endereco' com 'Endereço não informado' e cria 'idade_corrigida' preenchendo nulos com a média da 'idade'.
7.  **Tratamento de formato de dados**: Converte a coluna 'data' para o tipo datetime no formato dia/mês/ano, tratando erros com 'coerce'.
8.  **Tratamento de dados duplicados**: Remove linhas duplicadas, focando especificamente na duplicação pela coluna 'cpf'.
9.  **Salvamento dos dados limpos**: Cria um novo DataFrame com colunas selecionadas ('nome', 'cpf', 'idade', 'data', 'endereco', 'estado') e o salva em um novo arquivo CSV chamado 'clientes_limpeza.csv', sem o índice do DataFrame.
10. **Verificação do novo DataFrame**: Imprime o conteúdo do arquivo CSV salvo para confirmação.

In [ ]:
import pandas as pd

df = pd.read_csv('clientes.csv')

pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
print(df.head())
print(df.tail())
print(df.describe())

# Remover dados

df.drop('pais', axis=1, inplace=True) # Coluna
df.drop(2, axis=0, inplace=True) # Linha

# Normalizar campos de texto: geralmente existe um padrão, utiliza apenas 01, não mistura.
df['nome'] = df['nome'].str.title()
df['endereco'] = df['endereco'].str.lower()
df['estado'] = df['estado'].str.strip().str.upper()

# Converter tipos de dados
df['idade'] = df['idade'].astype(int)

print('Normalizar textos', df.head())

# Tratar valores nulos (ausentes)
df_fillna = df.fillna(0) # Substituir valores nulos po 0
df_dropna = df.dropna() # Remover registros com valores nulos
df_dropna4 = df.dropna(thresh=4) # Manter registro com no minimo 4 valores não nulos
df = df.dropna(subset=['cpf']) # Remover registro com CPF nulo

print('Valores nulos:\n', df.isnull().sum())
print('Qtd de registros nulos com fillna:', df_fillna.isnull().sum().sum())
print('Qtd de registros nulos com dropna:', df_dropna.isnull().sum().sum())
print('Qtd de registros nulos com dropna4:', df_dropna4.isnull().sum().sum())
print('Qtd de registros nulos com CPF:', df.isnull().sum().sum())

df.fillna({'estado': 'Desconhecido'}, inplace=True)
df['endereco'] = df['endereco'].fillna('Endereço não informado')
df['idade_corrigida'] = df['idade'].fillna(df['idade'].mean())

# Tratar formato de dados
df['data_corrigida'] = pd.to_datetime(df['data'], format='%d/%m/%Y', errors='coerce')

# Tratar dados duplicados
print('Qtd de registro atual:', df.shape[0])
df.drop_duplicates()
df.drop_duplicates(subset='cpf', inplace=True)
print('Qtd de registro removendo os duplicados:', len(df))

print('Dados Limpos:\n', df)

# Salvar dataframe
df['data'] = df['data_corrigida']
df['idade_corrigida'] = df['idade_corrigida']

df_salvar = df[['nome', 'cpf', 'idade', 'data', 'endereco', 'estado']]
df_salvar.to_csv('clientes_limpeza.csv', index=False)

print('Novo DataFrame:\n', pd.read_csv('clientes_limpeza.csv'))

### Explicação Detalhada do Código

Vamos analisar o código passo a passo:

```python
import pandas as pd
```
*   `import pandas as pd`: Importa a biblioteca Pandas, que é essencial para manipulação e análise de dados em Python. Ela é importada com o apelido `pd` para facilitar o uso.

```python
df = pd.read_csv('clientes.csv')
```
*   `df = pd.read_csv('clientes.csv')`: Lê o arquivo CSV chamado 'clientes.csv' e carrega seus dados em um DataFrame do Pandas. Um DataFrame é uma estrutura de dados tabular, semelhante a uma planilha ou tabela de banco de dados.

```python
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
```
*   `pd.set_option(...)`: Estas linhas configuram as opções de exibição do Pandas. Elas garantem que, ao imprimir um DataFrame, todas as colunas, linhas e o conteúdo completo das colunas sejam mostrados, sem truncamento, facilitando a visualização dos dados.

```python
print(df.head())
print(df.tail())
print(df.describe())
```
*   `print(df.head())`: Exibe as primeiras 5 linhas do DataFrame `df`, útil para uma visão rápida dos dados.
*   `print(df.tail())`: Exibe as últimas 5 linhas do DataFrame `df`, para verificar o final dos dados.
*   `print(df.describe())`: Gera estatísticas descritivas (contagem, média, desvio padrão, mínimo, máximo, quartis) para as colunas numéricas do DataFrame.

```python
df.drop('pais', axis=1, inplace=True) # Coluna
df.drop(2, axis=0, inplace=True) # Linha
```
*   `df.drop('pais', axis=1, inplace=True)`: Remove a coluna chamada 'pais' do DataFrame. `axis=1` indica que é uma coluna, e `inplace=True` faz a alteração diretamente no `df` sem precisar atribuir a uma nova variável.
*   `df.drop(2, axis=0, inplace=True)`: Remove a linha com o índice `2` do DataFrame. `axis=0` indica que é uma linha, e `inplace=True` aplica a alteração no `df`.

```python
df['nome'] = df['nome'].str.title()
df['endereco'] = df['endereco'].str.lower()
df['estado'] = df['estado'].str.strip().str.upper()
```
*   `df['nome'].str.title()`: Converte a primeira letra de cada palavra na coluna 'nome' para maiúscula e o restante para minúscula (formato de título).
*   `df['endereco'].str.lower()`: Converte todos os caracteres na coluna 'endereco' para minúsculas.
*   `df['estado'].str.strip().str.upper()`: Primeiro remove espaços em branco no início e fim das strings na coluna 'estado' (`.str.strip()`), e depois converte todos os caracteres para maiúsculas (`.str.upper()`).

```python
df['idade'] = df['idade'].astype(int)
```
*   `df['idade'].astype(int)`: Converte o tipo de dados da coluna 'idade' para inteiro. Isso é importante para garantir que idades sejam tratadas como números inteiros.

```python
print('Normalizar textos', df.head())
```
*   `print('Normalizar textos', df.head())`: Imprime um cabeçalho e as primeiras linhas do DataFrame após a normalização dos textos para verificar as alterações.

```python
df_fillna = df.fillna(0) # Substituir valores nulos po 0
df_dropna = df.dropna() # Remover registros com valores nulos
df_dropna4 = df.dropna(thresh=4) # Manter registro com no minimo 4 valores não nulos
df = df.dropna(subset=['cpf']) # Remover registro com CPF nulo
```
*   `df_fillna = df.fillna(0)`: Cria um novo DataFrame (`df_fillna`) onde todos os valores nulos (NaN) são substituídos por `0`.
*   `df_dropna = df.dropna()`: Cria um novo DataFrame (`df_dropna`) removendo *todas* as linhas que contêm pelo menos um valor nulo.
*   `df_dropna4 = df.dropna(thresh=4)`: Cria um novo DataFrame (`df_dropna4`) removendo as linhas que têm menos de 4 valores não nulos. Ou seja, mantém apenas as linhas com pelo menos 4 valores válidos.
*   `df = df.dropna(subset=['cpf'])`: Remove linhas do `df` original onde a coluna 'cpf' possui um valor nulo. A alteração é feita diretamente no `df`.

```python
print('Valores nulos:\n', df.isnull().sum())
print('Qtd de registros nulos com fillna:', df_fillna.isnull().sum().sum())
print('Qtd de registros nulos com dropna:', df_dropna.isnull().sum().sum())
print('Qtd de registros nulos com dropna4:', df_dropna4.isnull().sum().sum())
print('Qtd de registros nulos com CPF:', df.isnull().sum().sum())
```
*   `df.isnull().sum()`: Conta o número de valores nulos em cada coluna do DataFrame `df`.
*   `df_fillna.isnull().sum().sum()`: Calcula o número total de valores nulos no DataFrame `df_fillna`.
*   As linhas seguintes fazem o mesmo para `df_dropna`, `df_dropna4` e `df` (após a remoção baseada em CPF), para mostrar o efeito das operações de tratamento de nulos.

```python
df.fillna({'estado': 'Desconhecido'}, inplace=True)
df['endereco'] = df['endereco'].fillna('Endereço não informado')
df['idade_corrigida'] = df['idade'].fillna(df['idade'].mean())
```
*   `df.fillna({'estado': 'Desconhecido'}, inplace=True)`: Preenche valores nulos na coluna 'estado' com a string 'Desconhecido'.
*   `df['endereco'] = df['endereco'].fillna('Endereço não informado')`: Preenche valores nulos na coluna 'endereco' com a string 'Endereço não informado'.
*   `df['idade_corrigida'] = df['idade'].fillna(df['idade'].mean())`: Cria uma nova coluna 'idade_corrigida'. Se a 'idade' original tiver um valor nulo, ele é preenchido com a média das idades existentes na coluna 'idade'.

```python
df['data_corrigida'] = pd.to_datetime(df['data'], format='%d/%m/%Y', errors='coerce')
```
*   `pd.to_datetime(df['data'], format='%d/%m/%Y', errors='coerce')`: Converte a coluna 'data' para o formato de data e hora do Pandas (`datetime`). O `format='%d/%m/%Y'` especifica que as datas estão no formato dia/mês/ano. `errors='coerce'` fará com que qualquer data que não possa ser convertida para o formato especificado seja transformada em `NaT` (Not a Time), que é o valor nulo para datas.

```python
print('Qtd de registro atual:', df.shape[0])
df.drop_duplicates()
df.drop_duplicates(subset='cpf', inplace=True)
print('Qtd de registro removendo os duplicados:', len(df))
```
*   `df.shape[0]`: Retorna o número de linhas (registros) no DataFrame `df`.
*   `df.drop_duplicates()`: Esta linha *não* faz uma alteração inplace. Ela retornaria um novo DataFrame sem duplicatas, mas o resultado não é atribuído, então não afeta `df`.
*   `df.drop_duplicates(subset='cpf', inplace=True)`: Remove linhas duplicadas, mas apenas considera a coluna 'cpf' para identificar duplicatas. Se houver duas linhas com o mesmo CPF, uma delas será removida. `inplace=True` aplica a alteração diretamente no `df`.
*   `len(df)`: Retorna o número de linhas no DataFrame `df` após a remoção de duplicatas.

```python
print('Dados Limpos:\n', df)
```
*   `print('Dados Limpos:\n', df)`: Exibe o DataFrame completo após todas as operações de limpeza e tratamento.

```python
df['data'] = df['data_corrigida']
df['idade_corrigida'] = df['idade_corrigida']

df_salvar = df[['nome', 'cpf', 'idade', 'data', 'endereco', 'estado']]
df_salvar.to_csv('clientes_limpeza.csv', index=False)
```
*   `df['data'] = df['data_corrigida']`: Sobrescreve a coluna 'data' original com os valores da 'data_corrigida' (que está no formato `datetime`).
*   `df['idade_corrigida'] = df['idade_corrigida']`: Esta linha parece redundante, pois atribui a coluna a si mesma. É possível que o objetivo fosse renomear ou fazer alguma outra operação, mas como está, não altera a coluna.
*   `df_salvar = df[['nome', 'cpf', 'idade', 'data', 'endereco', 'estado']]`: Cria um novo DataFrame chamado `df_salvar` contendo apenas as colunas especificadas.
*   `df_salvar.to_csv('clientes_limpeza.csv', index=False)`: Salva o DataFrame `df_salvar` em um novo arquivo CSV chamado 'clientes_limpeza.csv'. `index=False` evita que o índice do DataFrame seja salvo como uma coluna no arquivo CSV.

```python
print('Novo DataFrame:\n', pd.read_csv('clientes_limpeza.csv'))
```
*   `print('Novo DataFrame:\n', pd.read_csv('clientes_limpeza.csv'))`: Lê o arquivo CSV recém-salvo ('clientes_limpeza.csv') e o imprime, confirmando que os dados foram salvos corretamente e que o novo arquivo pode ser lido.